In [138]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import mean_squared_error

# import dataset

In [ ]:
df = pd.read_csv('london_weather.csv')


# fill missing data

In [139]:
df = df.fillna(1)


# split dataset

In [149]:
x = df.drop(columns=['max_temp','snow_depth'], axis=1)  
y = df['max_temp']  

x = np.array(x)
y = np.array(y)

In [142]:
from sklearn.model_selection import train_test_split

x, x_test_main, y, y_test_main = train_test_split(x, y, test_size=0.1, random_state=42)
# y_train = y_train.astype(int)
# y_test = y_test.astype(int)
y_test_main.shape

(1535,)

# normalizing

In [143]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
x = scaler.fit_transform(x)
x_test_main = scaler.fit_transform(x_test_main)
x[0]

array([0.36609191, 0.33333333, 0.86875   , 0.82663317, 0.67213115,
       0.6568915 , 0.        , 0.98282754])

# pca

In [144]:
# from sklearn.decomposition import PCA

# pca = PCA(n_components=8)

# x = pca.fit_transform(X=x)
# x_test_main = pca.transform(X=x_test_main)


# define model 1

In [145]:


# model = keras.Sequential()
# model.add(keras.layers.Input(shape=(8,)))
# model.add(keras.layers.Dense(units=32, activation='relu'))
# model.add(keras.layers.Dense(units=32, activation='sigmoid'))
# model.add(keras.layers.Dense(units=1))
# model.compile(optimizer=tf.optimizers.Adam(), loss='mean_squared_error', metrics=['accuracy'])
# # model.compile(optimizer=tf.optimizers.Adam(), loss=tf.losses.mean_squared_error, metrics=['accuracy'])
# model.fit(x, y, epochs=20, batch_size=64, validation_data=(x_test_main, y_test_main))
# y_pred = model.predict(x_test_main)
# mse = mean_squared_error(y_true=y_test_main, y_pred=y_pred)
# mse
# # 12.187250703339979 => mean_squared_error with scaler and pca
# # 10.015396159320842 => mean_squared_error with scaler and without pca
# # 42.39169921336769 => mean_squared_error without scaler and without pca
# # 10.933645420023264 => mean_squared_error without column snow_depth and without pca and with scaler

# in simple model just do MinMaxScaler got the best result

# define 2

In [146]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=6)
# def model_creator():
#     model = keras.Sequential()
#     model.add(keras.layers.Input(shape=(8,)))
#     model.add(keras.layers.Dense(units=32, activation='relu'))
#     model.add(keras.layers.Dense(units=32, activation='relu'))
#     model.add(keras.layers.Dense(units=1, activation='linear'))
#     model.compile(optimizer=tf.optimizers.Adam(), loss='mean_squared_error', metrics=['accuracy'])
#     return model



# mse_list = []
# for train, validation in kf.split(x, y):

#     # x_train = x_train_main[train]
#     # y_train = y_train_main[train]

#     # x_test = x_train_main[validation]
#     # y_test = x_train_main[validation]

#     # x_train, x_test = x[train], x[validation]
#     # y_train, y_test = y[train], y[validation]

#     x_train = x[train]
#     x_test = x[validation]
#     y_train = y[train]
#     y_test = y[validation]
    
#     model = model_creator()
#     model.fit(x_train, y_train, epochs=20, batch_size=64, validation_data=(x_test, y_test))

#     y_pred = model.predict(x_test)

#     mse_list.append(mean_squared_error(y_test, y_pred))

# # mse_list

# y_pred_main = model.predict(x_test_main)
# mse_main = mean_squared_error(y_test_main, y_pred_main)
# mse_main, np.mean(mse_list)




# # 6.304961665296364 => mean_squared_error with scaler and without pca




# define 3

In [147]:
# from sklearn.model_selection import RandomizedSearchCV
# from keras.wrappers.scikit_learn import KerasClassifier

# # Define a function to create your Keras model
# def create_model(activation='relu', hidden_layer_size=32, dropout_rate=0.2):
#     # ... (Your model architecture)

#     return model

# # Create a KerasClassifier instance
# model = KerasClassifier(build_fn=create_model, verbose=0)

# # Define the hyperparameter search space
# param_dist = {
#     'activation': ['relu', 'tanh', 'sigmoid'],
#     'hidden_layer_size': [16, 32, 64, 128],
#     'dropout_rate': [0.2, 0.3, 0.4, 0.5]
# }

# # Create a RandomizedSearchCV object
# random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=10, cv=3)

# # Fit the random search to your data
# random_search.fit(X_train, y_train)

# # Get the best parameters and model
# best_params = random_search.best_params_
# best_model = random_search.best_estimator_

# define model 4

In [148]:
# def get_reg(meta, hidden_layer_sizes, dropout, active='relu'):
#     n_features_in_ = meta["n_features_in_"]
#     model = keras.models.Sequential()
#     model.add(keras.layers.Input(shape=(n_features_in_,)))
#     for hidden_layer_size in hidden_layer_sizes:
#         model.add(keras.layers.Dense(hidden_layer_size, activation=active))
#         model.add(keras.layers.Dropout(dropout))
#     model.add(keras.layers.Dense(1))
#     return model
# from scikeras.wrappers import KerasRegressor


# reg = KerasRegressor(
#     model=get_reg,
#     loss="mse",
#     metrics=[keras.metrics.R2Score],
#     hidden_layer_sizes=(100,),
#     dropout=0.5,
# )
# reg.fit(x_train, y_train)
# from sklearn.model_selection import RandomizedSearchCV

# param_grid = {
#     'alpha': [0.1, 1.0, 10.0, 100.0, 1000.0],
#     'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
# }
# rs = RandomizedSearchCV(estimator=reg, param_distributions=param_grid, cv=3, n_iter=20)

# rs.fit(x_train, y_train)
